<a href="https://colab.research.google.com/github/Alyssa-ckt/AI-Market-Research-Assistant/blob/main/AI_Market_Research_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -qU langchain-community wikipedia

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
# if we test with Gemini LLM
!pip install -q langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 1.4 MB/s eta 0:00:00


In [4]:
from langchain_community.retrievers import WikipediaRetriever

retriever = WikipediaRetriever(top_k_results = 10)

In [5]:
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd
import time
import os
from google.colab import userdata

In [6]:
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [8]:
# import packages
from langchain_google_genai import ChatGoogleGenerativeAI

# select model
model = "models/gemini-3-flash-preview"

# initialize model
max_output_tokens = 100
llm = ChatGoogleGenerativeAI(
    model=model,
    temperature=0,
    max_output_tokens=max_output_tokens,
)

In [10]:
# if we test with Llama
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 3.5 MB/s eta 0:00:00


In [11]:
from langchain_groq import ChatGroq
os.environ['GROQ_API_KEY'] = userdata.get('llama')

In [12]:
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0.5,
    max_tokens=1200,
    max_retries=2
)

In [13]:
# Testing the wiki retriever

query = "solar panels"

# 1. Fetch the documents
docs = retriever.invoke(query)

# 2. Extract and print (and store for the next step)
sources_list = []

if not docs:
    print("No Wikipedia results found.")
else:
    for i, doc in enumerate(docs, start=1):
        title = doc.metadata.get('title', 'Unknown Title')
        url = doc.metadata.get('source', 'No URL available')

        sources_list.append(f"Title: {title}\nURL: {url}\nContent: {doc.page_content[:500]}...") # Get a snippet

        print(f"{i}. {title}")
        print(f"   URL: {url}\n")

# 3. Join them for your AI prompt
context_for_ai = "\n\n".join(sources_list)

1. Solar panel
   URL: https://en.wikipedia.org/wiki/Solar_panel

2. Solar panels on spacecraft
   URL: https://en.wikipedia.org/wiki/Solar_panels_on_spacecraft

3. Agrivoltaics
   URL: https://en.wikipedia.org/wiki/Agrivoltaics

4. Solar energy
   URL: https://en.wikipedia.org/wiki/Solar_energy

5. Photovoltaics
   URL: https://en.wikipedia.org/wiki/Photovoltaics

6. Solar power in the United States
   URL: https://en.wikipedia.org/wiki/Solar_power_in_the_United_States

7. Solar power at the White House
   URL: https://en.wikipedia.org/wiki/Solar_power_at_the_White_House

8. Solar cell
   URL: https://en.wikipedia.org/wiki/Solar_cell

9. Solar power by country
   URL: https://en.wikipedia.org/wiki/Solar_power_by_country

10. Solar power
   URL: https://en.wikipedia.org/wiki/Solar_power



In [18]:
# Final version of the refined prompting and agent building trained using Llama LLM
from langchain_community.retrievers import WikipediaRetriever
from langchain_core.prompts import ChatPromptTemplate
import re

def generate_industry_report():
  while True:
    user_input = input("\nEnter industry: ").strip()

    if len(user_input) < 3:
      print("[INPUT ERROR] Please enter a valid industry name (e.g., 'Healthcare', 'Cosmetics').")
      continue

    if not user_input or not re.search(r"[a-zA-Z]", user_input):
      print("[INPUT REQUIRED] Please enter a valid industry (e.g., 'Healthcare', 'Semiconductors').")
      return

    industry_check_prompt = f"""
    You are an input validator.

    TASK:
    Determine whether the "{user_input}" refers to a BUSINESS INDUSTRY, SECTOR, or MARKET.

    Respond with EXACTLY one word:
    VALID or INVALID

    Then give a one sentence reason if it is INVALID
    """
    classification_raw = llm.invoke(industry_check_prompt).content.strip()
    classification = classification_raw.split()[0].upper()

    if classification == "VALID":
        break
    else:
        print(f"'{user_input}' does not appear to be a valid industry.")
        reason = classification_raw.replace("INVALID", "").strip()
        if reason:
            print(f"Reason: {reason}")

  # --- STEP 1: MULTI-QUERY EXPANSION ---
  setup_prompt = f"""
  You are a research query planner for a market research assistant.

  INDUSTRY:
  "{user_input}"

  OBJECTIVE:
  Generate Wikipedia search queries that will retrieve pages useful for producing
  a business-focused industry research report.

  STRICT RULES:
  - Every query MUST explicitly refer to the "{user_input}" industry or a very close synonym.
  - Do NOT generate queries for adjacent or parent industries unless they explicitly include "{user_input}".
  - Prefer canonical Wikipedia article titles over descriptive phrases.
  - Avoid geography-only, company-only, or historical-only queries.
  - Do NOT include future predictions or forecasts unless Wikipedia commonly covers them.

  TASK:
  Generate exactly 5 Wikipedia search queries, each targeting a distinct business dimension:

  1. Industry definition and scope
  2. Market structure and value chain
  3. Competitive landscape and key players
  4. Economic significance or market size (if available)
  5. Industry trends, regulation, or structural change

  FORMAT (EXACT):
  QUERIES:
  - <query 1>
  - <query 2>
  - <query 3>
  - <query 4>
  - <query 5>
  """

  setup_data = llm.invoke(setup_prompt).content.strip()

  queries = [
      q.strip("- ").strip()
      for q in setup_data.splitlines()
      if q.strip().startswith("-")
  ]

  retriever = WikipediaRetriever(load_max_docs=17, lang="en")

  all_docs = []
  for q in queries:
      all_docs.extend(retriever.invoke(q))

  # Deduplicate
  unique_docs = {doc.metadata["title"]: doc for doc in all_docs}
  raw_docs = list(unique_docs.values())

  if not raw_docs:
      print("No results found.")
      return

  # --- STEP 2: BOUNCER ---
  titles_list = [doc.metadata["title"] for doc in raw_docs]

  bouncer_prompt = f"""
  You are a source quality filter for a business market research tool.

  INDUSTRY:
  "{user_input}"

  TASK:
  From the list below, select only 5 Wikipedia article titles that:
  - Describe the "{user_input}" industry as a whole
  - Are useful for business or market analysis
  - NOT country-specific, company-specific, or historical-only

  RETURN FORMAT:
  - Return ONLY the exact titles, separated by commas.
  - Do NOT explain your reasoning.

  CANDIDATE TITLES:
  {titles_list}
  """

  verified_titles = [
      t.strip().lower()
      for t in llm.invoke(bouncer_prompt).content.split(",")
  ]

  final_docs = [
      doc for doc in raw_docs
      if any(v in doc.metadata["title"].lower() for v in verified_titles)
  ]

  # Deduplicate by title first
  seen = set()
  deduped_docs = []

  for doc in final_docs:
      title = doc.metadata["title"]
      if title not in seen:
          seen.add(title)
          deduped_docs.append(doc)

  final_docs = deduped_docs[:5]

  final_docs = [doc for doc in raw_docs if any(v in doc.metadata["title"].lower() for v in verified_titles)]
  seen_titles = {doc.metadata["title"] for doc in final_docs}

  # Fill missing slots
  if len(final_docs) < 5:
    print("[!] Expanding search scope...")
    broad_queries = [
        user_input,
        f"{user_input} sector",
        f"{user_input} services",
        f"{user_input} market"
        ]

    # Search each broad query and add results
    for q in broad_queries:
      docs = retriever.invoke(q)
      for doc in docs:
        title = doc.metadata["title"]
    # Add only if it's new and contains the industry keyword
        if title not in [d.metadata["title"] for d in final_docs]:
            if user_input.lower() in title.lower():  # strict relevance
                final_docs.append(doc)

  # Cap at 5 sources
  final_docs = final_docs[:5]

  if not final_docs:
      print("\n[!] DATA GAP DETECTED")
      print(f"I couldn't find specific business market data for '{user_input}' on Wikipedia.")
      print("Please provide more details (e.g., 'Automotive manufacturing' instead of just 'Cars') or try a different sector.")
      return # This stops the function immediately so no report is generated

  # STEP 3: Context Preparation & Report
  context_text = ""
  print("-" * 15 + " VERIFIED SOURCES " + "-" * 15)

  for i, doc in enumerate(final_docs, 1):
      title = doc.metadata.get('title')
      url = doc.metadata.get('source')
      print(f"{i}. {title}\n   URL: {url}")

      # Build the context block for the report
      context_text += (
          f"[SOURCE {i}]\n"
          f"TITLE: {title}\n"
          f"URL: {url}\n"
          f"CONTENT:\n{doc.page_content[:1500]}\n\n"
          )

  # Final Generation
  report_prompt = ChatPromptTemplate.from_template("""
  ROLE:
  You are a Market Research Assistant supporting business analysts at a large corporation.

  OBJECTIVE:
  Produce a concise, decision-oriented industry briefing that helps a corporate analyst
  understand the structure, economics, risks, and strategic outlook of the {user_input} industry.

  STRICT RULES:
  - Use ONLY information explicitly contained in the CONTEXT below.
  - Every factual statement MUST end with at least one clickable citation in this format: [SOURCE X](URL).
  - Do NOT include assumptions, estimates, or forward-looking claims unless directly supported by the sources.
  - Avoid generic business statements that could apply to most industries.
  - If a topic is not covered in the sources, explicitly state: "Not covered in sources."

  WRITING STYLE:
  - Professional, neutral, and analytical.
  - Focus on industry mechanisms, not definitions.
  - Prioritize insights that matter for corporate strategy and risk assessment.

  REPORT STRUCTURE:

  1. Industry Overview
    Briefly describe what the industry does, its core economic function, and why it matters
    in the broader economy (2–3 sentences).

  2. Market Structure & Value Chain
    Explain how value is created and captured in the industry.
    Identify key participants (e.g. producers, intermediaries, regulators, customers)
    and how they interact.

  3. Industry Scale & Geographic Footprint
    Describe the industry’s global or regional scale, major operating regions,
    and any notable geographic concentration patterns.

  4. Competitive Landscape
    Identify major players and explain how competition is structured
    (e.g. fragmentation, concentration, role of incumbents vs new entrants).

  5. Key Industry Drivers
    List 3–4 concrete drivers that influence industry performance.
    Drivers should reflect structural, regulatory, technological, or demand-side forces
    specific to the {user_input} industry.

  6. Risks, Constraints & Regulatory Barriers
    Describe the most significant risks and constraints facing the industry,
    including regulatory requirements, cost structures, operational risks,
    or external pressures.

  7. SWOT Analysis
    Provide an industry-specific SWOT summary:
    - Strengths: Structural or economic advantages unique to the industry
    - Weaknesses: Structural limitations or inefficiencies
    - Opportunities: Changes or trends that could materially improve industry performance
    - Threats: External or internal forces that could materially harm the industry
    Each point must be grounded in the provided sources and avoid generic statements.

  8. Industry Outlook
    Summarize how the industry is expected to evolve based on current trends
    described in the sources (2–3 sentences).

  LENGTH:
  Maximum 500 words.

  CONTEXT:
  {context}
  """)

  report = (report_prompt | llm).invoke({
      "context": context_text,
      "user_input": user_input
  })


  print("\n" + "="*60 + f"\nREPORT: {user_input.upper()}\n" + "="*60 + f"\n{report.content}")

  # This tells Python to execute the function when you run the script
if __name__ == "__main__":
    generate_industry_report()


Enter industry: plastic
--------------- VERIFIED SOURCES ---------------
1. Plastics industry
   URL: https://en.wikipedia.org/wiki/Plastics_industry
2. Plastic
   URL: https://en.wikipedia.org/wiki/Plastic
3. Plastic bag bans in the United States
   URL: https://en.wikipedia.org/wiki/Plastic_bag_bans_in_the_United_States
4. Polypropylene
   URL: https://en.wikipedia.org/wiki/Polypropylene
5. High-density polyethylene
   URL: https://en.wikipedia.org/wiki/High-density_polyethylene

REPORT: PLASTIC
**Industry Overview**
The plastic industry manufactures polymer materials and offers services in plastics important to various industries, including packaging, building and construction, electronics, aerospace, manufacturing, and transportation [SOURCE 1](https://en.wikipedia.org/wiki/Plastics_industry). The industry is part of the chemical and petrochemical industries, with mineral oil being a major constituent of plastics [SOURCE 1](https://en.wikipedia.org/wiki/Plastics_industry). The ind

In [19]:
# First draft using OpenAI LLM (however, ran out of tokens to train the model on; hence final refinement is based on the Llama LLM)

llm = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", temperature=0.2, max_retries=5)

def generate_industry_report():
    user_input = input("Enter the industry for research: ").strip()

    # ONE CALL for both Validation and Expansion
    setup_prompt = f"""
    Analyze the term: '{user_input}'
    1. Is this a valid industry sector? (YES/NO)
    2. If YES, provide a 3-word Wikipedia search query to get a strategic overview.
    Format: VALID: [YES/NO] | QUERY: [Query]
    """
    setup_response = llm.invoke(setup_prompt).text.strip()

    if "VALID: NO" in setup_response:
        print("Invalid industry. Please try again.")
        return generate_industry_report()

    refined_query = setup_response.split("QUERY: ")[1]
    print(f"Refining search to: {refined_query}")

    # Step 2: Search
    search_results = wikipedia.search(refined_query, results=5)
    context_text = ""
    for title in search_results:
        try:
            page = wikipedia.page(title, auto_suggest=False)
            context_text += f"SOURCE: {page.url}\nCONTENT: {page.content[:3000]}\n\n"
        except: continue

    # --- STEP 3: Strategic Report Generation (< 500 words) ---
    print("\n[STEP 3] Synthesizing strategic report...")
    prompt = ChatPromptTemplate.from_template("""
    ROLE: Senior Business Analyst.
    TASK: Write a strategic industry report based ONLY on the provided Wikipedia context.
    Use professional 'Investment' terminology and cite URL headings for each section.
    LENGTH: Strictly under 500 words.

    STRUCTURE:
    1. Executive Summary: Current industry state.
    2. Market Pillars: Key technologies or economic drivers (cite URLs).
    3. Strategic Risks: Barriers to entry or obsolescence threats.
    4. Data Gaps: What critical business info is missing from these sources?

    CONTEXT:
    {context}
    """)

    chain = prompt | llm
    report = chain.invoke({"context": context_text})

    print("\n" + "="*50)
    print(f"MARKET RESEARCH REPORT: {user_input.upper()}")
    print("="*50 + "\n")
    print(report.text)

# Run the assistant
if __name__ == "__main__":
    generate_industry_report()

Enter the industry for research: plastic
Refining search to: Plastics industry overview

[STEP 3] Synthesizing strategic report...

MARKET RESEARCH REPORT: PLASTIC

**To:** Investment Committee
**From:** Senior Business Analyst
**Subject:** Strategic Analysis of the Global Plastics and Chemical Industrial Complex

### 1. Executive Summary
The global plastics industry is characterized by aggressive production scaling and deep integration into essential economic sectors. As of 2023, annual production exceeded 400 million metric tons, with a projected trajectory to surpass 1.3 billion tons by 2060. This growth is fueled by the material’s low-cost production, durability, and high utility in packaging (40% of usage) and construction (20%). While the sector remains a cornerstone of the modern economy, it faces a systemic "sustainability deficit," with historical recycling rates stagnating at approximately 9%.

### 2. Market Pillars

**Material Science and Industrial Utility**
*(Source: https